In [ ]:
import plotly.express as px
import seaborn as sns
import numpy as np
from tqdm.auto import trange

In [ ]:
import numpy as np
from scipy.stats import binom, poisson

In [ ]:
a = np.arange(80,82)
b = np.arange(500,503)
d = np.arange(400,404)

In [ ]:
params = {"amin": 75, "amax": 90, "bmin": 500, "bmax": 600, "p1": 0.1, "p2": 0.01, "p3": 0.3}

cmin = dmin = 0
cmax = params["amax"] + params["bmax"]
dmax = 2 * (params["amax"] + params["bmax"])

a_sup = np.arange(params["amin"], params["amax"] + 1)
b_sup = np.arange(params["bmin"], params["bmax"] + 1)
c_sup = np.arange(cmin, cmax + 1)
d_sup = np.arange(dmin, dmax + 1)

In [ ]:
def pa(params, model):
    a_vals = np.arange(params['amin'], params['amax'] + 1)
    pa_ = np.full(a_vals.shape, 1 / a_vals.size)
    return pa_, a_vals

In [ ]:
def pb(params, model):
    b_vals = np.arange(params['bmin'], params['bmax'] + 1)
    pb_ = np.full(b_vals.shape, 1 / b_vals.size)
    return pb_, b_vals

In [ ]:
def pc_ab(a, b, params, model):
    c_vals = np.arange(0, params["amax"] + params["bmax"] + 1)
    pc_ab_ = np.zeros((c_vals.size, a.size, b.size))
    if model == 1:
        bin_a = binom.pmf(c_vals[:, None], a[None, :], params["p1"])
        bin_b = binom.pmf(c_vals[:, None], b[None, :], params["p2"])
        for i in c_vals:
            pc_ab_[i] = bin_a[: i + 1].T @ bin_b[: i + 1][::-1]
    else:
        lambda_ = a[:, None] * params["p1"] + b * params["p2"]
        pc_ab_ = poisson.pmf(c_vals[:, None, None], lambda_)
    return pc_ab_, c_vals

In [ ]:
def pc_a(a, params, model):
    pb_, b_vals = pb(params, model)
    pc_ab_, c_vals = pc_ab(a, b_vals, params, model)
    pc_a_ = np.einsum('kij,j->ki', pc_ab_, pb_)
    return pc_a_, c_vals

In [ ]:
def pc_b(b, params, model):
    pa_, a_vals = pa(params, model)
    pc_ab_, c_vals = pc_ab(a_vals, b, params, model)
    pc_b_ = np.einsum('kij,i->kj', pc_ab_, pa_)
    return pc_b_, c_vals

In [ ]:
def pc(params, model):
    pa_, a_vals = pa(params, model)
    pb_, b_vals = pb(params, model)
    pc_ab_, c_vals = pc_ab(a_vals, b_vals, params, model)
    pc_ = np.einsum('ijk,j,k->i', pc_ab_, pa_, pb_)
    pc_ /= np.sum(pc_)
    return pc_, c_vals

In [ ]:
def pd_c(c, params, model):
    d_vals = np.arange(2 * (params["amax"] + params["bmax"]) + 1)
    x = d_vals[:, None] - c[None, :]
    pd_c_ = binom.pmf(x, c, params["p3"])
    return pd_c_, d_vals

In [ ]:
def pd(params, model):
    pc_, c_vals = pc(params, model)
    pd_c_, d_vals = pd_c(c_vals, params, model)
    pd_ = np.einsum('ij,j->i', pd_c_, pc_)
    pd_ /= np.sum(pd_)
    return pd_, d_vals

In [ ]:
def pc_d(d, params, model):
    pc_, c_vals = pc(params, model)
    pd_c_, d_vals = pd_c(c_vals, params, model)
    pc_d_ = pd_c_[d] * pc_
    pc_d_ /= np.sum(pc_d_, axis=1, keepdims=True)
    return pc_d_.T, c_vals

In [ ]:
def pc_abd(a, b, d, params, model):
    pc_ab_, c_vals = pc_ab(a, b, params, model)
    pd_c_, d_vals = pd_c(c_vals, params, model)
    pc_abd_ = pd_c_[d][:,:, None, None] * pc_ab_[None, :, :, :]
    pc_abd_ /= np.einsum('ij,jkl->ikl', pd_c_[d], pc_ab_)[:, None, :, :]
    return pc_abd_.transpose(1, 2, 3, 0), c_vals

In [ ]:
def expectation(probs, vals):
    return probs.T @ vals

def variance(probs, vals):
    return expectation(probs, vals ** 2) - expectation(probs, vals) ** 2

In [ ]:
a.shape

(2,)

In [ ]:
Ea_arr = np.full((1, ), 83)
Eb_arr = np.full((1, ), 550)
Ed_arr = np.full((1, ), 18)

In [ ]:
import plotly.express as px

# Данные
model = 1
pc_, x = pc(params, model)
pc_a_, _ = pc_a(Ea_arr, params, model)
pc_b_, _ = pc_b(Eb_arr, params, model)
pc_d_, _ = pc_d(Ed_arr, params, model)
pc_ab_, _ = pc_ab(Ea_arr, Eb_arr, params, model)
pc_abd_, _ = pc_abd(Ea_arr, Eb_arr, Ed_arr, params, model)

cut = 25  # Уменьшенный cut до 25
x_vals = x[:cut]

# Создаем фигуру
fig = px.line()

# Добавляем линии для каждого распределения с настройкой прозрачности и толщины
fig.add_scatter(x=x_vals, y=pc_[:cut], mode='lines', name='p(c)', line=dict(width=2))
fig.add_scatter(x=x_vals, y=pc_a_.flatten()[:cut], mode='lines', name='p(c|a)', line=dict(width=2, dash='dash'))
fig.add_scatter(x=x_vals, y=pc_b_.flatten()[:cut], mode='lines', name='p(c|b)', line=dict(width=2, dash='dot'))
fig.add_scatter(x=x_vals, y=pc_d_.flatten()[:cut], mode='lines', name='p(c|d)', line=dict(width=2, dash='dashdot'))
fig.add_scatter(x=x_vals, y=pc_ab_.flatten()[:cut], mode='lines', name='p(c|ab)', line=dict(width=2, dash='solid'))
fig.add_scatter(x=x_vals, y=pc_abd_.flatten()[:cut], mode='lines', name='p(c|abd)', line=dict(width=2, dash='longdash'))

# Настройка осей и подписи
fig.update_layout(
    xaxis_title="c",
    yaxis_title="$\\rho$",
    title="Распределения",
    legend_title="Распределение",
    width=700,  # Уменьшенная ширина графика
    height=600  # Высота графика
)

# Отображаем график в ноутбуке
fig.show()


In [ ]:
model = 2
Ec = expectation(*pc(params, model))
Dc = variance(*pc(params, model))
print(f"E[c] = {Ec} | D[c] = {Dc}")

pc_a_ = pc_a(Ea_arr, params, model)
Ec_a = np.mean(expectation(*pc_a_))
Dc_a = np.mean(variance(*pc_a_))
print(f"E[c|a] = {Ec_a} | D[c|a] = {Dc_a}")

pc_b_ = pc_b(Eb_arr, params, model)
Ec_b = np.mean(expectation(*pc_b_))
Dc_b = np.mean(variance(*pc_b_))
print(f"E[c|b] = {Ec_b} | D[c|b] = {Dc_b}")

pc_d_ = pc_d(Ed_arr, params, model)
Ec_d = expectation(*pc_d_).mean()
Dc_d = variance(*pc_d_).mean()
print(f"E[c|d] = {Ec_d} | D[c|d] = {Dc_d}")

pc_ab_ = pc_ab(Ea_arr, Eb_arr, params, model)
Ec_ab = expectation(*pc_ab_).mean()
Dc_ab = variance(*pc_ab_).mean()
print(f"E[c|ab] = {Ec_ab} | D[c|ab] = {Dc_ab}")

pc_abd_ = pc_abd(Ea_arr, Eb_arr, Ed_arr, params, model)
Ec_abd = expectation(*pc_abd_).mean()
Dc_abd = variance(*pc_abd_).mean()
print(f"E[c|abd] = {Ec_abd} | D[c|abd] = {Dc_abd}")

E[c] = 13.75 | D[c] = 14.047500000000014
E[c|a] = 13.799999999999992 | D[c|a] = 13.885000000000105
E[c|b] = 13.749999999999988 | D[c|b] = 13.962500000000091
E[c|d] = 13.893833892236302 | D[c|d] = 1.5439433290956117
E[c|ab] = 13.800000000000011 | D[c|ab] = 13.799999999999898
E[c|abd] = 13.900175167039475 | D[c|abd] = 1.5408838495819452


In [ ]:
import artem_kotov_v1 as code

In [ ]:
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)
d = np.arange(0, 2 * (params["amax"] + params["bmax"]) + 1)

Dc_d1 = code.variance(*code.pc_d(d, params, 1))
Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))


/content/artem_kotov_v1.py:126: RuntimeWarning: invalid value encountered in divide
  prob /= np.sum(prob, axis=1, keepdims=True)


In [ ]:
Dc_d2 = code.variance(*code.pc_d(d, params, 2))
Dc_a2 = code.variance(*code.pc_a(a, params, 2))
Dc_b2 = code.variance(*code.pc_b(b, params, 2))


In [ ]:
np.nanmax(Dc_d1), np.nanmin(Dc_a1), np.nanmax(Dc_b1)

(10.298690905139665, 12.279999999999973, 13.577500000000214)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Функция для построения графика модели 1 и модели 2
def plot_variance_comparison(Dc_d, Dc_a, Dc_b, model_number):
    fig = go.Figure()

    # Добавляем линии для каждой метрики
    fig.add_trace(go.Scatter(x=d, y=Dc_d, mode='lines', name="D[c|d]", line=dict(color='green', width=2)))
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_a), np.min(Dc_a)], mode='lines', name="min(D[c|a])", line=dict(color='red', dash='dash')))
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_b), np.min(Dc_b)], mode='lines', name="min(D[c|b])", line=dict(color='yellow', dash='dash')))

    # Настройка осей и подписи
    fig.update_layout(
        title=f"Модель {model_number}",
        xaxis_title="d values",
        yaxis_title="D[c|x]",
        width=600,
        height=400
    )

    return fig

# Строим два графика для двух моделей
fig1 = plot_variance_comparison(Dc_d1, Dc_a1, Dc_b1, 1)
fig2 = plot_variance_comparison(Dc_d2, Dc_a2, Dc_b2, 2)

# Отображаем графики
fig1.show()
fig2.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Данные
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)
d = np.arange(0, 2 * (params["amax"] + params["bmax"]) + 1)

Dc_d1 = code.variance(*code.pc_d(d, params, 1))[:458]
Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

Dc_d2 = code.variance(*code.pc_d(d, params, 2))[:521]
Dc_a2 = code.variance(*code.pc_a(a, params, 2))
Dc_b2 = code.variance(*code.pc_b(b, params, 2))

# Функция для построения графика модели 1 и модели 2 с измененными цветами и шрифтами
def plot_variance_comparison(Dc_d, Dc_a, Dc_b, model_number):
    fig = go.Figure()

    # Добавляем линии для каждой метрики с более заметными цветами
    fig.add_trace(go.Scatter(x=d, y=Dc_d, mode='lines', name="D[c|d]", line=dict(color='lime', width=3)))  # Яркий зеленый
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_a), np.min(Dc_a)], mode='lines', name="min(D[c|a])", line=dict(color='red', dash='dash', width=2)))  # Красный
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_b), np.min(Dc_b)], mode='lines', name="min(D[c|b])", line=dict(color='orange', dash='dash', width=2)))  # Оранжевый

    # Настройка осей и подписи с черным шрифтом
    fig.update_layout(
        title=dict(text=f"Модель {model_number}", font=dict(size=20, color='black')),
        xaxis_title=dict(text="d values", font=dict(size=16, color='black')),
        yaxis_title=dict(text="D[c|x]", font=dict(size=16, color='black')),
        font=dict(size=14, color='black'),  # Общие настройки шрифта
        width=600,
        height=400,
        plot_bgcolor='white',  # Белый фон
        paper_bgcolor='white'
    )

    # Настройка осей с черными линиями
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', zerolinecolor='black', linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', zerolinecolor='black', linecolor='black')

    return fig

# Строим два графика для двух моделей
fig1 = plot_variance_comparison(Dc_d1, Dc_a1, Dc_b1, 1)
fig2 = plot_variance_comparison(Dc_d2, Dc_a2, Dc_b2, 2)

# Отображаем графики
fig1.show()
fig2.show()


/content/artem_kotov_v1.py:126: RuntimeWarning:

invalid value encountered in divide



In [ ]:
import plotly.graph_objects as go
import numpy as np

# Функция для построения графика с дефолтным светло-голубым фоном и контрастными цветами линий
def plot_variance_comparison(Dc_d, Dc_a, Dc_b, model_number):
    fig = go.Figure()

    # Добавляем линии для каждой метрики с контрастными цветами
    if model_number == 1:

        fig.add_trace(go.Scatter(x=d[:458], y=Dc_d, mode='lines', name="D[c|d]", line=dict(color='blue', width=3)))  # Синий
    else:
        fig.add_trace(go.Scatter(x=d[:521], y=Dc_d, mode='lines', name="D[c|d]", line=dict(color='blue', width=3)))  # Синий
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_a), np.min(Dc_a)], mode='lines', name="min(D[c|a])", line=dict(color='purple', width=2)))  # Зеленый
    fig.add_trace(go.Scatter(x=[d[0], d[-1]], y=[np.min(Dc_b), np.min(Dc_b)], mode='lines', name="min(D[c|b])", line=dict(color='red', width=2)))  # Красный

    # Настройка осей и подписи с черным шрифтом
    fig.update_layout(
        title=dict(text=f"Модель {model_number}", font=dict(size=20, color='black')),
        xaxis_title=dict(text="d values", font=dict(size=16, color='black')),
        yaxis_title=dict(text="D[c|x]", font=dict(size=16, color='black')),
        font=dict(size=14, color='black', family="Times New Roman"),  # Общие настройки шрифта
        width=600,
        height=400,
        xaxis=dict(range=[0, 550])
    )

    return fig

# Строим два графика для двух моделей
fig1 = plot_variance_comparison(Dc_d1, Dc_a1, Dc_b1, 1)
fig2 = plot_variance_comparison(Dc_d2, Dc_a2, Dc_b2, 2)

# Отображаем графики
fig1.show()
fig2.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Данные для графика
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)

Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

Dc_a2 = code.variance(*code.pc_a(a, params, 2))
Dc_b2 = code.variance(*code.pc_b(b, params, 2))

# Логика распределения точек для первой модели
left_1, right_1 = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b1[b_] < Dc_a1[a_]:
            left_1.append((a_, b_))
        else:
            right_1.append((a_, b_))

# Логика распределения точек для второй модели
left_2, right_2 = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b2[b_] < Dc_a2[a_]:
            left_2.append((a_, b_))
        else:
            right_2.append((a_, b_))

# Преобразуем в numpy массивы
left_1 = np.array(left_1)
right_1 = np.array(right_1)

left_2 = np.array(left_2)
right_2 = np.array(right_2)

# Создаем подграфики: первый слева, второй справа
fig = make_subplots(rows=1, cols=2, subplot_titles=("Модель 1", "Модель 2"))

# График для первой модели
fig.add_trace(go.Scatter(x=left_1[:, 0], y=left_1[:, 1], mode='markers', name="D[c|b] < D[c|a]", marker=dict(size=5, color='#6F2DA8')),
              row=1, col=1)
fig.add_trace(go.Scatter(x=right_1[:, 0], y=right_1[:, 1], mode='markers', name="D[c|b] >= D[c|a]", marker=dict(size=5, color='#E55451')),
              row=1, col=1)

# График для второй модели
fig.add_trace(go.Scatter(x=left_2[:, 0], y=left_2[:, 1], mode='markers', name="D[c|b] < D[c|a]", marker=dict(size=5, color='#6F2DA8')),
              row=1, col=2)
fig.add_trace(go.Scatter(x=right_2[:, 0], y=right_2[:, 1], mode='markers', name="D[c|b] >= D[c|a]", marker=dict(size=5, color='#E55451')),
              row=1, col=2)

# Настройка шрифта и внешнего вида
fig.update_layout(
    font=dict(size=20, family="Times New Roman", color="black"),
    width=1000,
    height=1100,
    showlegend=True,
    title_text="Точки, удовлетворяющие обозначенным множествам"
)

# Настройка осей
fig.update_xaxes(title_text="a", row=1, col=1)
fig.update_yaxes(title_text="b", row=1, col=1)

fig.update_xaxes(title_text="a", row=1, col=2)
fig.update_yaxes(title_text="b", row=1, col=2)

# Показать график
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Данные для графика
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)

Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

Dc_a2 = code.variance(*code.pc_a(a, params, 2))
Dc_b2 = code.variance(*code.pc_b(b, params, 2))

# Логика распределения точек для первой модели
left_1, right_1 = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b1[b_] < Dc_a1[a_]:
            left_1.append((a_, b_))
        else:
            right_1.append((a_, b_))

# Логика распределения точек для второй модели
left_2, right_2 = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b2[b_] < Dc_a2[a_]:
            left_2.append((a_, b_))
        else:
            right_2.append((a_, b_))

# Преобразуем в numpy массивы
left_1 = np.array(left_1)
right_1 = np.array(right_1)

left_2 = np.array(left_2)
right_2 = np.array(right_2)

# Создаем подграфики: первый слева, второй справа
fig = make_subplots(rows=1, cols=2, subplot_titles=("Модель 1", "Модель 2"))

# График для первой модели
fig.add_trace(go.Scatter(x=left_1[:, 0], y=left_1[:, 1], mode='markers', name="D[c|b] < D[c|a]", marker=dict(size=6, color='blue')),
              row=1, col=1)
fig.add_trace(go.Scatter(x=right_1[:, 0], y=right_1[:, 1], mode='markers', name="D[c|b] >= D[c|a]", marker=dict(size=6, color='red')),
              row=1, col=1)

# График для второй модели
fig.add_trace(go.Scatter(x=left_2[:, 0], y=left_2[:, 1], mode='markers', name="D[c|b] < D[c|a]", marker=dict(size=6, color='blue')),
              row=1, col=2)
fig.add_trace(go.Scatter(x=right_2[:, 0], y=right_2[:, 1], mode='markers', name="D[c|b] >= D[c|a]", marker=dict(size=6, color='red')),
              row=1, col=2)

# Настройка шрифта и внешнего вида с Times New Roman
fig.update_layout(
    font=dict(size=16, family="Times New Roman", color="black"),
    width=1200,  # Увеличенная ширина графика
    height=900,  # Увеличенная высота графика
    showlegend=False,
    title_text="Точки, удовлетворяющие обозначенным множествам"
)

# Настройка осей для первого графика
fig.update_xaxes(title_text="a", row=1, col=1, range=[a.min(), a.max()])
fig.update_yaxes(title_text="b", row=1, col=1, range=[b.min(), b.max()])

# Настройка осей для второго графика
fig.update_xaxes(title_text="a", row=1, col=2, range=[a.min(), a.max()])
fig.update_yaxes(title_text="b", row=1, col=2, range=[b.min(), b.max()])

# Показать график
fig.show()


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import plotly.express as px

# Данные для двух моделей (примерно такие же, как в предыдущих задачах)
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)

Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

Dc_a2 = code.variance(*code.pc_a(a, params, 2))
Dc_b2 = code.variance(*code.pc_b(b, params, 2))

# Создаем метки классов (1 для первой модели, 2 для второй модели)
labels1 = np.ones(len(Dc_a1))
labels2 = np.zeros(len(Dc_a2))

# Объединяем данные и метки
a_total = np.concatenate([Dc_a1, Dc_a2])
b_total = np.concatenate([Dc_b1, Dc_b2])
labels_total = np.concatenate([labels1, labels2])

# Формируем массив признаков (a, b)
X = np.column_stack([a_total, b_total])

# Модель логистической регрессии
log_reg = LogisticRegression()
log_reg.fit(X, labels_total)

# Получаем предсказания
predictions = log_reg.predict(X)

# Визуализация
fig = px.scatter(x=a_total, y=b_total, color=predictions, labels={"color": "Модель"})
fig.update_layout(title="Результаты логистической регрессии", xaxis_title="a", yaxis_title="b")
fig.show()


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 32 and the array at index 1 has size 202

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import plotly.express as px

# Предположим, что у нас уже есть массивы a и b для модели
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)

# Рассчитываем дисперсии для модели 1
Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

# Рассчитываем множества left и right для модели 1
left, right = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b1[b_] < Dc_a1[a_]:
            left.append((a_, b_))  # Класс 0
        else:
            right.append((a_, b_))  # Класс 1

# Преобразуем множества в массивы
left = np.array(left)
right = np.array(right)

# Объединяем данные
X = np.vstack([left, right])
y = np.concatenate([np.zeros(len(left)), np.ones(len(right))])  # Метки классов (0 для left, 1 для right)

# Построение логистической регрессии
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Получаем предсказания
predictions = log_reg.predict(X)

# Визуализация
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=predictions, labels={"color": "Классы"})
fig.update_layout(title="Результаты логистической регрессии для модели 1", xaxis_title="a", yaxis_title="b")
fig.show()


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import plotly.express as px

a = np.arange(0, params["amax"] + 1)
b = np.arange(0, params["bmax"] + 1)

# Рассчитываем дисперсии для модели 1
Dc_a1 = code.variance(*code.pc_a(a, params, 2))
Dc_b1 = code.variance(*code.pc_b(b, params, 2))

# Рассчитываем множества left и right для модели 1
left, right = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b1[b_] < Dc_a1[a_]:
            left.append((a_, b_))  # Класс 0
        else:
            right.append((a_, b_))  # Класс 1

# Преобразуем множества в массивы
left = np.array(left)
right = np.array(right)

# Объединяем данные
X = np.vstack([left, right])
y = np.concatenate([np.zeros(len(left)), np.ones(len(right))])  # Метки классов (0 для left, 1 для right)

# Построение логистической регрессии
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Получаем коэффициенты разделяющей плоскости
coef = log_reg.coef_[0]  # Коэффициенты для a и b
intercept = log_reg.intercept_[0]  # Свободный член

# Функция для разделяющей плоскости (b = - (coef[0] * a + intercept) / coef[1])
def separating_line(a_vals, coef, intercept):
    return -(coef[0] * a_vals + intercept) / coef[1]

# Получаем значения для разделяющей линии
a_vals = np.linspace(0, a.max() + 1)
b_vals = separating_line(a_vals, coef, intercept)

# Визуализация
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y, labels={"color": "Классы"})

# Добавляем линию разделяющей плоскости
fig.add_scatter(x=a_vals, y=b_vals, mode="lines", name="Разделяющая плоскость", line=dict(color="black"))

# Обновляем заголовки и оформление
fig.update_layout(
    title="Результаты логистической регрессии для модели 1 с разделяющей плоскостью",
    xaxis_title="a",
    yaxis_title="b"
)

fig.show()

# Для второй модели аналогичный код:
# Используйте Dc_a2 и Dc_b2 вместо Dc_a1 и Dc_b1 для модели 2.


In [ ]:
log_reg.coef_[0],log_reg.intercept_[0]

(array([-10.01054377,   1.00108132]), 287.79386268900697)

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import plotly.express as px

# Данные
a = np.arange(params["amin"], params["amax"] + 1)
b = np.arange(params["bmin"], params["bmax"] + 1)

Dc_a1 = code.variance(*code.pc_a(a, params, 1))
Dc_b1 = code.variance(*code.pc_b(b, params, 1))

# Создаем множества точек (left и right) для разделения
left, right = [], []
for a_ in range(a.size):
    for b_ in range(b.size):
        if Dc_b1[b_] < Dc_a1[a_]:
            left.append((a_, b_))  # Класс 0
        else:
            right.append((a_, b_))  # Класс 1

left = np.array(left)
right = np.array(right)

# Объединяем данные для логистической регрессии
X = np.vstack([left, right])
y = np.concatenate([np.zeros(len(left)), np.ones(len(right))])  # Метки классов

# Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Получаем коэффициенты для уравнения разделяющей плоскости
coef = log_reg.coef_[0]  # Коэффициенты для a и b
intercept = log_reg.intercept_[0]  # Свободный член

# Функция для получения разделяющей плоскости
def separating_line(a_vals, coef, intercept):
    return -(coef[0] * a_vals + intercept) / coef[1]

# Значения для оси 'a'
a_vals = np.linspace(a.min(), a.max(), 100)
# Значения для разделяющей линии 'b'
b_vals = separating_line(a_vals, coef, intercept)

# Визуализация точек классов и разделяющей плоскости
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=y, labels={"color": "Классы"})

# Добавляем разделяющую линию
fig.add_scatter(x=a_vals, y=b_vals, mode="lines", name="Разделяющая плоскость", line=dict(color="black"))

# Настройка осей и заголовков
fig.update_layout(
    title="Логистическая регрессия: Модель 1",
    xaxis_title="a",
    yaxis_title="b",
    xaxis=dict(range=[a.min(), a.max()]),  # Фиксируем диапазон оси 'a'
    yaxis=dict(range=[b.min(), b.max()])   # Фиксируем диапазон оси 'b'
)

# Показываем график
fig.show()

# Аналогично для модели 2:
# Замените Dc_a1 и Dc_b1 на Dc_a2 и Dc_b2 для второй модели
